In [1]:
from smach_utils2 import *



In [3]:
fp_msg = FindPersonRequest()
pc = rospy.wait_for_message("/hsrb/head_rgbd_sensor_depth_registered/rectified_points", PointCloud2)
pc = fp_msg.cloud
resp = clothes_color(fp_msg)